In [35]:
import sys
# sys.path.append(r'C:\Users\edwar\Documents\LabStuff\ibllib\brainbox\io')
#sys.path.append(r'C:\Users\IBL\Documents\ibl_analysis\ONE\one')
import matplotlib.pyplot as plt
import numpy as np
from brainbox.io.one import load_wheel_reaction_times
import brainbox.behavior.wheel as wh
from ibllib.io.raw_data_loaders import *
from ibllib.pipes.tasks import *
from ibllib.pipes import *
from ibllib.io.extractors.bpod_trials import *
from one.api import ONE
from one.alf.io import *
from ibllib.io.extractors.camera import CameraTimestampsBpod
from ibllib.io.extractors.ephys_fpga import extract_wheel_moves
from ibllib.io.extractors.training_wheel import extract_first_movement_times
#from localAlf import load_wheel_reaction_times_local
from brainbox.behavior.training import *

from ScanImageTiffReader import ScanImageTiffReader
import json
import ast
import math

In [36]:
session_path = r'Z:\TM_Lab\Edward\Data\iblrig_data\Subjects\Q_R2\2024-04-02\001'
try:
    temp_trial_data = load_object(session_path+'\\alf', 'trials')
except:
    pass
trials, wheel, output_files = extract_all(session_path, save=True)

2024-04-18 15:20:59.209 WARNING  [raw_data_loaders.py:655] _iblrig_encoderPositions.raw.ssv rotary encoder timestamps swapped at index: 40030  Z:\TM_Lab\Edward\Data\iblrig_data\Subjects\Q_R2\2024-04-02\001\raw_behavior_data\_iblrig_encoderPositions.raw.ssv


In [37]:
img_path = r'Z:\TM_Lab\Edward\Cerebellum_Imaging\Q_R2\20240402\acq1\acq1_2roi_15hz_00001.tif'
reader=ScanImageTiffReader(img_path)
frames_metadata = []

with ScanImageTiffReader(img_path) as reader:
    num_frames = reader.shape()[0]  # Assuming the first dimension represents frames
    
    for i in range(num_frames):
        description = reader.description(i)
        frame_metadata = {}  # Dictionary to hold this frame's metadata
        
        for line in description.split('\n'):
            if '=' in line:  # Ensure there's a key-value pair to parse
                key, value = line.split('=', 1)  # Split by the first '='
                key = key.strip()
                
                try:
                    # Safely evaluate the value string to the appropriate Python data type
                    # This works for numbers, lists, dictionaries, etc.
                    value = ast.literal_eval(value.strip())
                except (ValueError, SyntaxError):
                    # If evaluation fails, keep the value as a string
                    value = value.strip()
                
                frame_metadata[key] = value
        
        frames_metadata.append(frame_metadata)


In [38]:
auxTrigger1 = []
frame_time = []
# Iterate through each frame's metadata
for frame_metadata in frames_metadata:
    frame_time.append(frame_metadata['frameTimestamps_sec'])
    # Check if 'auxTrigger0' exists and is not empty
    if 'auxTrigger1' in frame_metadata and frame_metadata['auxTrigger1']:
        # Extend the all_auxTrigger0 list with the numbers from this frame's auxTrigger0
        auxTrigger1.extend(frame_metadata['auxTrigger1'])

In [53]:
frames_metadata[436]

{'frameNumbers': 437,
 'acquisitionNumbers': 1,
 'frameNumberAcquisition': 437,
 'frameTimestamps_sec': 29.091134645,
 'acqTriggerTimestamps_sec': -1.0,
 'nextFileMarkerTimestamps_sec': -1.0,
 'endOfAcquisition': 0,
 'endOfAcquisitionMode': 0,
 'dcOverVoltage': 0,
 'epoch': '[2024  4  2 16 58 19.182]',
 'auxTrigger0': [],
 'auxTrigger1': [29.138187415],
 'auxTrigger2': [],
 'auxTrigger3': [],
 'I2CData': {}}

In [39]:
combined_list = trials['goCueTrigger_times'] + trials['errorCueTrigger_times']


# Concatenate arrays
combined_array = np.concatenate((trials['goCueTrigger_times'], trials['errorCueTrigger_times']))

# Remove nan values
cleaned_array = combined_array[~np.isnan(combined_array)]

# Sort the array
sorted_array = np.sort(cleaned_array)
print(len(sorted_array))
print(len(auxTrigger1))

223
223


In [40]:
print(f'first go cue in image: {auxTrigger1[0]}')
print(f"first go cue in statemachine table: {trials['table']['goCue_times'][0]}")
print(f"first go cue in trial data: {trials['goCueTrigger_times'][0]}")

first go cue in image: 12.07188692
first go cue in statemachine table: 5.368099999999998
first go cue in trial data: 5.170200000000001


In [41]:
img_offset = auxTrigger1[0]-sorted_array[0] # time in s that img is leading the first goCueTime
statemachine_offset = auxTrigger1[0]-trials['table']['goCue_times'][0] # time in s that state machine is leading the first goCueTime


In [42]:
trials['table']

,intervals_0,intervals_1,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,feedback_times,feedbackType,rewardVolume,probabilityLeft,firstMovement_times
0,0.000000,20.270901,5.368100,17.708800,1,5.401400,NaN,0.5,17.921700,-1,0.0,0.5,11.787196
1,21.619699,77.504702,22.434299,75.941399,1,22.217899,0.5,NaN,75.941499,1,3.0,0.9,39.790196
2,78.917099,132.055201,79.848699,130.494699,1,79.635299,0.5,NaN,130.494799,1,3.0,0.5,99.097196
3,133.462099,139.988601,135.685899,138.427199,-1,135.468899,NaN,0.5,138.427299,1,3.0,0.5,138.045196
4,141.261000,146.868301,142.202700,144.296100,-1,141.987400,0.5,NaN,144.521000,-1,0.0,0.5,143.126196
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,2930.132998,2946.782300,2930.876798,2944.234198,1,2930.665598,NaN,0.5,2944.442598,-1,0.0,0.5,2932.995196
154,2948.009799,2990.909900,2948.742099,2989.358099,-1,2948.527099,NaN,0.5,2989.358199,1,3.0,0.3,NaN
155,2992.252198,3012.855200,2993.074898,3011.255098,-1,2992.866098,NaN,1.0,3011.255198,1,3.0,0.5,2994.971196
156,3014.101499,3026.183101,3014.826599,3024.633899,-1,3014.616199,NaN,1.0,3024.633999,1,3.0,0.5,3023.448196


In [43]:
np.diff(np.array(trials['table']['goCue_times']))

array([17.066199, 57.4144  , 55.8372  ,  6.516801,  7.865999, 15.6991  ,
       11.6655  ,  6.720801,  5.282699,  5.3977  ,  5.9498  ,  3.9695  ,
        8.132   , 20.0816  ,  5.187701,  4.747599,  7.2839  ,  5.817801,
        6.330799,  7.0329  , 14.1528  , 16.165101,  7.482   , 15.2009  ,
       12.917699,  8.369101,  6.285799, 24.328101,  5.237699,  5.065701,
        5.997699, 10.1653  ,  4.0996  , 39.137   , 14.317801, 28.647699,
       62.702101,  9.569199, 14.843   , 28.7686  , 26.062401, 31.953099,
       14.6839  , 38.323   , 30.8139  ,  9.734301,  6.881899,  6.054701,
        4.370599, 10.4603  , 28.3187  , 64.7603  , 15.534   , 25.636301,
       10.550399, 56.550301, 13.295699, 10.1543  , 38.0669  , 47.7331  ,
       20.264601, 45.089899, 37.4758  , 48.8033  ,  5.217601, 10.581399,
        6.7129  ,  6.8579  ,  5.2116  ,  6.1838  , 28.4667  ,  6.1308  ,
       19.5205  , 42.6495  ,  4.1705  ,  6.3278  , 19.2865  ,  9.2182  ,
       22.6319  , 21.7988  ,  7.234001, 13.361699, 

In [45]:
idx = 0
ts = np.array(trials['table']['goCue_times'])
fb = np.array(trials['table']['feedbackType'])
new_ts = np.array([])
for i in range(len(ts)-1):
    if fb[i] == 1:
        new_ts = np.append(new_ts, np.diff(auxTrigger1)[idx])
        idx+=1
    else:
        new_ts = np.append(new_ts, np.diff(auxTrigger1)[idx]+np.diff(auxTrigger1)[idx+1])
        idx+=2


In [46]:
new_ts

array([17.0663005 , 57.41478673, 55.83755474,  6.51687951,  7.86606191,
       15.69913024, 11.66557798,  6.72089908,  5.28270999,  5.39773484,
        5.9498066 ,  3.96954423,  8.13209172, 20.08171978,  5.1877143 ,
        4.74763671,  7.2839957 ,  5.81779485,  6.33085671,  7.03294398,
       14.15293043, 16.16518839,  7.48201918, 15.20104873, 12.91777046,
        8.36912994,  6.28585179, 24.3283103 ,  5.23770442,  5.06569755,
        5.99779816, 10.16538788,  4.09955976, 39.13729342, 14.31794123,
       28.64789076, 62.70249318,  9.56930229, 14.84301706, 28.76888815,
       26.062542  , 31.95331625, 14.6839809 , 38.32321476, 30.81417022,
        9.7343215 ,  6.88192939,  6.05482649,  4.37058758, 10.46041003,
       28.31885469, 64.76076245, 15.53410179, 25.63648372, 10.55043106,
       56.55065555, 13.29579694, 10.15437467, 38.0671582 , 47.73345031,
       20.26475299, 45.09010785, 37.47606217, 48.80360452,  5.21770548,
       10.58143027,  6.71291641,  6.85793408,  5.21168639,  6.18

In [47]:
table_for_img = trials['table'].copy()

In [49]:
def find_closest_index(value, list_of_values):
    # Find the index of the closest value in list_of_values
    return np.abs(np.array(list_of_values) - value).argmin()

table_for_img = trials['table'].copy()
for col in table_for_img.columns:
    if 'times' in col:
        table_for_img[col] = table_for_img[col].add(statemachine_offset)
        table_for_img[col] = table_for_img[col].apply(lambda x: find_closest_index(x, frame_time))

In [50]:
table_for_img

,intervals_0,intervals_1,goCue_times,response_times,choice,stimOn_times,contrastLeft,contrastRight,feedback_times,feedbackType,rewardVolume,probabilityLeft,firstMovement_times
0,0.000000,20.270901,181,366,1,181,NaN,0.5,369,-1,0.0,0.5,277
1,21.619699,77.504702,437,1239,1,433,0.5,NaN,1239,1,3.0,0.9,697
2,78.917099,132.055201,1297,2056,1,1294,0.5,NaN,2056,1,3.0,0.5,1585
3,133.462099,139.988601,2134,2175,-1,2130,NaN,0.5,2175,1,3.0,0.5,2169
4,141.261000,146.868301,2231,2263,-1,2228,0.5,NaN,2266,-1,0.0,0.5,2245
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,2930.132998,2946.782300,44012,44212,1,44009,NaN,0.5,44215,-1,0.0,0.5,44044
154,2948.009799,2990.909900,44280,44888,-1,44277,NaN,0.5,44888,1,3.0,0.3,0
155,2992.252198,3012.855200,44944,45216,-1,44941,NaN,1.0,45216,1,3.0,0.5,44972
156,3014.101499,3026.183101,45270,45417,-1,45267,NaN,1.0,45417,1,3.0,0.5,45399


In [54]:
table_for_img.to_csv(session_path+'\\table_for_img.csv', index=False)

In [14]:
camera_bin_filepath = r'Z:\TM_Lab\Edward\Data\iblrig_data\Subjects\O_L3\2024-03-26\003\raw_video_data\_iblrig_leftCamera.frameData.bin'
with open('file.bin', 'rb') as file:
    binary_data = file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'file.bin'